# Setup

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool
from pydantic import BaseModel

/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
# Define the LLM
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.5)

# Data Inputs

In [4]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [5]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

# Research

In [6]:
# Build a Research Agent
research_agent = Agent(
    role = "Influencer researcher",
    goal = f"Research about {youtube_lead['name']}'s channel {youtube_lead['channel_url']} to assess their suitability for {brand_info['name']}",
    backstory = f"You are an expert in Influencer partnerships that align with {brand_info['name']} and its respective values: {brand_info['brand_values']}",
    llm = llm,
    tools = [SerperDevTool(), ScrapeWebsiteTool(), YoutubeChannelSearchTool(), YoutubeVideoSearchTool()]
)

/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [7]:
# Define the google research task
channel_research_task = Task(
    description = f"""
    Research the channel {youtube_lead['name']} at {youtube_lead['channel_url']}:
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with {brand_info['name']}""",
    expected_output = "A research report on the channel's key attributes and audience",
    agent = research_agent
)

In [8]:
# Define the research videos task
videos_research_task = Task(
    description = f"""
    Research the most popular videos from channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Summarise their topics and analyse the audiences.""",
    expected_output="A research report on the most popular videos and their topics",
    agent = research_agent)

In [9]:
# Define a sentiment on the channel via Google searches
sentiment_task = Task(
    description = f"""
    Conduct a sentiment analysis via Google Search on the channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Look for user feedbacks, reviews, or discussions that indicate general sentiment.
    Search for instance on reddit.""",
    expected_output= "A sentiment analysis report on the channel's",
    agent = research_agent)

# AI Scoring Agent

Evaluate the potential partnership based on brand values and based on YouTube channels, videos and Google search.

In [10]:
# Developping the Scoring Agent
scoring_agent =  Agent(
    role = "Partnership Scoring Agent",
    goal = f"""Score the partnership between {brand_info['name']} and {youtube_lead['name']}.
    Assess based on the {brand_info['brand_values']}""",
    backstory = f"You are an expert in Influencer Marking working for {brand_info['name']}",
    llm = llm,
)

In [11]:
# Create a Base Model Class to provide a score
class Score(BaseModel):
    score: int
    justification: str

In [12]:
# Define the Scoring Task
scoring_task = Task(
    description = f"""
    Score the partnership between {brand_info['name']} and {youtube_lead['name']}""",
    expected_output = "A score between 1 and 10 and justification for the scoring. An explanation of the score indicating what fits the brand and what does not",
    agent = scoring_agent,
    context = [channel_research_task, videos_research_task, sentiment_task],
    output_pydantic= Score
)


# Build the AI Crew

In [13]:
# Assemble the AI crew
crew = Crew(
    agents = [research_agent, scoring_agent],
    tasks = [channel_research_task, videos_research_task, sentiment_task, scoring_task],
    process = Process.sequential,
    verbose = True
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0fc651a1-99f0-490e-9a34-d5b7024745be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bc310c8d-7b8e-4eff-bb97-45293c90bb01
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bc310c8d-7b8e-4eff-bb97-45293c90bb01
       Status: Executing Task...
    └── 🤖 Agent: Influencer researcher
            Status: In Progress

# Agent: Influencer researcher
## Task: 
    Research the channel Theo - t3․gg at https://www.youtube.com/@t3dotgg:
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with Zero to Mastery


🤖 Agent: Influencer researcher
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Partnership Scoring Agent
## Task: 
    Score the partnership between Zero to Mastery and Theo - t3․gg


# Agent: Partnership Scoring Agent
## Final Answer: 
{
  "score": 8,
  "justification": "The partnership between Zero to Mastery and Theo - t3․gg scores an 8 out of 10. Theo's YouTube channel aligns well with Zero to Mastery's core values of quality education, community-driven growth, career-focused learning, and up-to-date tech stacks. His focus on full stack TypeScript and modern web development practices resonates with Zero to Mastery's audience, which is interested in enhancing their skills and staying updated with current trends. The channel's significant subscriber count and high engagement levels indicate a strong potential for reaching a wide audience. However, some criticisms exist regarding Theo's communication style and perceived influence by sponsorships, which slightly detract from the overall alignment. Despite these concerns, the partnership remains highly be

🚀 Crew: crew
├── 📋 Task: bc310c8d-7b8e-4eff-bb97-45293c90bb01
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: c3176c03-86e8-4ffc-bdea-34007b4b0855
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: 1fc08a19-797e-4e8a-80f3-a64a5f2fa537
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
└── 📋 Task: 1b28b553-5ca6-4982-91aa-21d9021532b5
       Status: Executing Task...
    └── 🤖 Agent: Partnership Scoring Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: bc310c8d-7b8e-4eff-bb97-45293c90bb01
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: c3176c03-86e8-4ffc-bdea-34007b4b0855
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: 1fc08a19-797e-4e8a-80f3-a64a5f2fa537
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
└── 📋 Task: 1b28b553-5ca6-4982-91aa-21d9021532b5
       Assigned to: Partnership Scoring Agent
       Status: ✅ Completed
    └── 🤖 Agent: Partnership Scoring Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1b28b553-5ca6-4982-91aa-21d9021532b5                                                                     │
│  Agent: Partnership Scoring Agent                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0fc651a1-99f0-490e-9a34-d5b7024745be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:

import json
# Accessing the final outputs
channel_research_output = result.tasks_output[0].raw
videos_research_output = result.tasks_output[1].raw
sentiment_output = result.tasks_output[2].raw
scoring_output = result.tasks_output[3].raw

print("Channel Research Output:")
display(Markdown(channel_research_output))

print("\nVideos Research Output:")
display(Markdown(videos_research_output))

print("\nSentiment Analysis Output:")
display(Markdown(sentiment_output))

print("\nScoring Output:")
display(Markdown(scoring_output))

Channel Research Output:


Research Report on Theo - t3․gg YouTube Channel

1. Channel Focus and Content Type:
   - Theo - t3․gg is a YouTube channel primarily focused on software development, with a strong emphasis on full stack TypeScript. The channel provides a mix of insights, opinions, and discussions related to software development, industry trends, technology comparisons, and career advice.

2. Subscriber Count and Engagement:
   - The channel has approximately 420,000 to 425,000 subscribers, indicating a significant following. The channel's content has accumulated over 67 million views, reflecting high engagement and interest from the audience.

3. Audience Alignment with Zero to Mastery:
   - Given the channel's focus on quality education and career-focused learning in software development, particularly with up-to-date tech stacks like TypeScript, there is a strong alignment with Zero to Mastery's values of providing quality education, fostering community-driven growth, and focusing on career advancement in tech.

Overall, Theo - t3․gg appears to be a suitable candidate for partnership with Zero to Mastery, given its alignment with the core values and audience interests of the platform.


Videos Research Output:


Research Report on Theo - t3․gg YouTube Channel

1. Channel Focus and Content Type:
   - Theo - t3․gg is a YouTube channel primarily focused on software development, with a strong emphasis on full stack TypeScript. The channel provides a mix of insights, opinions, and discussions related to software development, industry trends, technology comparisons, and career advice.

2. Subscriber Count and Engagement:
   - The channel has approximately 420,000 to 425,000 subscribers, indicating a significant following. The channel's content has accumulated over 67 million views, reflecting high engagement and interest from the audience.

3. Most Popular Videos and Their Topics:
   - "PirateSoftware is right, this needs to stop": A video discussing industry practices and how they impact developers.
   - "The REAL Cost Of AWS (And How To Avoid It)": A detailed look into AWS costs and strategies to minimize them.
   - "From 0 to Production - The Modern React Tutorial (RSCs, ...": A comprehensive tutorial on using modern React features for production.
   - "If YouTube Devs Were Honest (featuring Theo @t3dotgg)": A humorous take on the realities of YouTube development.
   - "The Coolest Thing I Ever Built": An exploration of a personal project, showcasing development skills and innovation.
   - "Flip with Theo t3dotgg - TypeScript Type Challenges #4179 [MEDIUM]": A video tackling TypeScript challenges, aimed at intermediate developers.
   - "T3 Stack Tutorial - FROM 0 TO PROD FOR $0 (Next.js, ...": A tutorial on building a production-ready stack using Next.js.

4. Audience Analysis:
   - The audience mainly consists of software developers and tech enthusiasts interested in TypeScript, AWS, and modern web development practices. The channel's focus on practical tutorials and industry insights attracts viewers looking to enhance their skills and stay updated with current trends.

5. Audience Alignment with Zero to Mastery:
   - Given the channel's focus on quality education and career-focused learning in software development, particularly with up-to-date tech stacks like TypeScript, there is a strong alignment with Zero to Mastery's values of providing quality education, fostering community-driven growth, and focusing on career advancement in tech.

Overall, Theo - t3․gg appears to be a suitable candidate for partnership with Zero to Mastery, given its alignment with the core values and audience interests of the platform.


Sentiment Analysis Output:


1. **The Theo Problem : r/webdev**: Some users feel that Theo struggles with communication, making his content occasionally hard to take in. [Reddit Link](https://www.reddit.com/r/webdev/comments/17bi852/the_theo_problem/)

2. **Dev influencers who say they're honest: I don't believe it.**: Theo's style of videos is appreciated for being genuine and engaging, especially for mid and senior engineers. [Reddit Link](https://www.reddit.com/r/webdev/comments/17pw3am/dev_influencers_who_say_theyre_honest_i_dont/)

3. **I think Theo has made some mistakes in his rails demo**: Some criticism is noted, with a user claiming Theo tries too hard to be an influencer and that some of his content is paid. [Reddit Link](https://www.reddit.com/r/rails/comments/1gsosdy/i_think_theo_has_made_some_mistakes_in_his_rails/)

4. **What YouTubers should I subscribe to to learn front end ...**: Theo is recommended alongside others for beginners, but with a cautionary note about potential demotivation. [Reddit Link](https://www.reddit.com/r/Frontend/comments/179ka8r/what_youtubers_should_i_subscribe_to_to_learn/)

5. **Your favorite youtube channels for React videos/tutorials?**: Theo is seen more as an entertainer who can occasionally teach something rather than a mentor. [Reddit Link](https://www.reddit.com/r/reactjs/comments/16zvw20/your_favorite_youtube_channels_for_react/)

6. **What (tech)YouTubers are you following and why?**: Some users don't agree with many things Theo says but find his streams informative. [Reddit Link](https://www.reddit.com/r/developersIndia/comments/11c65nj/what_techyoutubers_are_you_following_and_why/)

7. **Why Theo is wrong & we'll get a Laravel for JavaScript**: Theo's content is compared unfavorably to others, suggesting a negative impression. [Reddit Link](https://www.reddit.com/r/javascript/comments/1d3d98w/why_theo_is_wrong_well_get_a_laravel_for/)

Overall, the sentiment is mixed, with appreciation for his engaging style and informative content, but also criticism regarding his communication and some content being seen as clickbait or influenced by sponsorships.


Scoring Output:


{
  "score": 8,
  "justification": "The partnership between Zero to Mastery and Theo - t3․gg scores an 8 out of 10. Theo's YouTube channel aligns well with Zero to Mastery's core values of quality education, community-driven growth, career-focused learning, and up-to-date tech stacks. His focus on full stack TypeScript and modern web development practices resonates with Zero to Mastery's audience, which is interested in enhancing their skills and staying updated with current trends. The channel's significant subscriber count and high engagement levels indicate a strong potential for reaching a wide audience. However, some criticisms exist regarding Theo's communication style and perceived influence by sponsorships, which slightly detract from the overall alignment. Despite these concerns, the partnership remains highly beneficial given the shared goals and audience interests."